# Face Recognition vs Multi Modal Analysis 

# Face Recognition Testing

In [99]:
def test_face_emotion_detection(csv_path):
    test_data = pd.read_csv(csv_path)

    results = []
    for idx, row in test_data.iterrows():
        try:
            # Face emotion detection
            face_emotion = detect_face_emotion_from_file(row['ImagePath'])
            face_emotion_readable = {k: round(float(v), 2) for k, v in face_emotion.items()}
            
            # Map face emotion results
            mapped_face = map_emotion_values(face_emotion, emotion_mappings['face'])
            final_emotion = max(mapped_face, key=mapped_face.get)
            
            results.append({
                "Record": idx + 1,
                "Ground Truth": row['GroundTruthEmotion'],
                #"Face Model": face_emotion_readable,
                "Predicted Emotion": final_emotion
            })
        except Exception as e:
            results.append({
                "Record": idx + 1,
                "Ground Truth": row['GroundTruthEmotion'],
                #"Face Model": "Error",
                "Predicted Emotion": "Error"
            })

    # Convert results to a DataFrame for a better display
    df_results = pd.DataFrame(results)

    # Display Results in a Clean Tabular Format
    print(tabulate(
        df_results,
        headers="keys",
        tablefmt="grid",
        showindex=False
    ))

    # Calculate and Display Accuracy
    valid_results = df_results[df_results["Predicted Emotion"] != "Error"]
    accuracy = accuracy_score(valid_results["Ground Truth"], valid_results["Predicted Emotion"])
    print(f"\nFinal Accuracy: {accuracy:.2f}")


# Run the function with the updated logic
#initialize_models()  # Ensure models are loaded
face_emotion_model = tf.keras.models.load_model(r'C:/Users/reeva/Desktop/690/best_model.keras')
csv_path = 'test_data - Copy.csv'  # Replace with your actual CSV path
print("Face Recognition Output::")
test_face_emotion_detection(csv_path)


Face Recognition Output::
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
+----------+----------------+---------------------+
|   Recor

# Multi Modal Integrated Module Testing

In [100]:
    import warnings
    import pandas as pd
    import numpy as np
    import cv2
    import torch
    import torch.nn as nn
    import tensorflow as tf
    import joblib
    from sklearn.metrics import accuracy_score
    from tabulate import tabulate
    
    warnings.filterwarnings("ignore")  # Suppress all warnings
    
    
    # Initialize Models
    def initialize_models():
        global face_emotion_model, lstm_model, sleep_model
    
        # Load face emotion model
        try:
            face_emotion_model = tf.keras.models.load_model(r'C:/Users/reeva/Desktop/690/best_model.keras')
            print("Face emotion model loaded successfully.")
        except Exception as e:
            print(f"Error loading face_emotion_model: {e}")
            face_emotion_model = None
    
        # Load LSTM model
        try:
            input_size = 2
            hidden_size = 64
            num_layers = 2
            num_classes = 4
            lstm_model = EmotionLSTM(input_size, hidden_size, num_layers, num_classes)
            checkpoint = torch.load(r'C:/Users/reeva/Desktop/690/best_emotion_model.pth', map_location=torch.device('cpu'))
            lstm_model.load_state_dict(checkpoint['model_state_dict'])
            lstm_model.eval()
            print("LSTM model loaded successfully.")
        except Exception as e:
            print(f"Error loading lstm_model: {e}")
            lstm_model = None
    
        # Load sleep model
        try:
            sleep_model = joblib.load(r'C:/Users/reeva/Desktop/690/sleep-edf-database-expanded-1.0.0-20241127T213628Z-001/sleep-edf-database-expanded-1.0.0/multi_label_model.joblib')
            print("Sleep emotion model loaded successfully.")
        except Exception as e:
            print(f"Error loading sleep_model: {e}")
            sleep_model = None
    
    
    # LSTM Model Definition
    class EmotionLSTM(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.5):
            super(EmotionLSTM, self).__init__()
            self.lstm = nn.LSTM(
                input_size, hidden_size, num_layers,
                batch_first=True, bidirectional=True,
                dropout=dropout if num_layers > 1 else 0
            )
            self.attention = nn.Sequential(
                nn.Linear(hidden_size * 2, 1),
                nn.Tanh()
            )
            self.dropout = nn.Dropout(dropout)
            self.fc = nn.Linear(hidden_size * 2, num_classes)
    
        def forward(self, x):
            lstm_out, _ = self.lstm(x)
            attention_weights = self.attention(lstm_out)
            attention_weights = torch.softmax(attention_weights, dim=1)
            context = torch.sum(attention_weights * lstm_out, dim=1)
            out = self.dropout(context)
            out = self.fc(out)
            return out
    
    
    # Helper Functions
    def map_emotion_values(module_values, mapping):
        common_emotions = ['Happy', 'Sad', 'Neutral', 'Angry', 'Relaxed', 'Stressed', 'Bored']
        mapped_values = {emotion: 0.0 for emotion in common_emotions}
    
        for emotion, value in module_values.items():
            if emotion in mapping:
                common_emotion = mapping[emotion]
                mapped_values[common_emotion] += value
            else:
                print(f"Debug: Unmapped emotion detected -> {emotion}. Defaulting to 'Neutral'.")
                mapped_values['Neutral'] += value  # Default to 'Neutral' for unmapped
    
        #print(f"Mapped Values: {mapped_values}")  # Debug log
        return mapped_values
    
    
    def combine_emotion_values(mapped_values_list, weights):
        combined_values = {emotion: 0.0 for emotion in mapped_values_list[0].keys()}
        for mapped_values, weight in zip(mapped_values_list, weights):
            for emotion, value in mapped_values.items():
                combined_values[emotion] += value * weight
    
        total = sum(combined_values.values())
        if total > 0:
            combined_values = {k: v / total for k, v in combined_values.items()}
        else:
            print("Debug: Combined values sum to 0.")  # Debug log
            combined_values['Neutral'] = 1.0
    
        #print(f"Combined Values: {combined_values}")  # Debug log
        return combined_values
    
    
    
    def detect_face_emotion_from_file(image_path):
        if not face_emotion_model:
            return {'Neutral': 1.0}
        try:
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Could not load image at {image_path}")
                return {'Neutral': 1.0}
    
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            roi_gray = cv2.resize(gray, (48, 48), interpolation=cv2.INTER_AREA)
            roi_rgb = np.expand_dims(np.stack([roi_gray] * 3, axis=-1) / 255.0, axis=0)
            predictions = face_emotion_model.predict(roi_rgb)[0]
            detected_emotions = dict(zip(['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'], predictions))
            #print(f"Face Detection Output: {detected_emotions}")  # Debug log
            return detected_emotions
        except Exception as e:
            print(f"Error during face emotion detection: {e}")
            return {'Neutral': 1.0}
    
    
    def detect_lstm_emotion(input_data):
        if not lstm_model:
            return {'Baseline': 1.0}
        try:
            tensor_data = torch.tensor(input_data, dtype=torch.float32).unsqueeze(0)
            output = lstm_model(tensor_data)
            probabilities = torch.softmax(output, dim=1).detach().numpy()[0]
            detected_emotions = dict(zip(['Baseline', 'Stress', 'Amusement', 'Meditation'], probabilities))
            #print(f"LSTM Detection Output: {detected_emotions}")  # Debug log
            return detected_emotions
        except Exception as e:
            print(f"Error during LSTM emotion detection: {e}")
            return {'Baseline': 1.0}
    
    def detect_sleep_emotion(input_data):
        """
        Simple heuristic-based sleep emotion detection.
        """
        try:
            # Example heuristic: Use the mean and range of the input data
            mean_value = np.mean(input_data)
            max_value = np.max(input_data)
            min_value = np.min(input_data)
    
            # Assign emotions based on heuristic
            if mean_value > 0.7:
                return {'happiness': 1.0, 'sadness': 0.0, 'anger': 0.0, 'surprise': 0.0, 'fear': 0.0}
            elif max_value - min_value > 0.5:
                return {'happiness': 0.0, 'sadness': 0.0, 'anger': 1.0, 'surprise': 0.0, 'fear': 0.0}
            elif mean_value < 0.3:
                return {'happiness': 0.0, 'sadness': 1.0, 'anger': 0.0, 'surprise': 0.0, 'fear': 0.0}
            else:
                return {'happiness': 0.0, 'sadness': 0.0, 'anger': 0.0, 'surprise': 0.0, 'fear': 1.0}
        except Exception as e:
            print(f"Error during sleep emotion detection heuristic: {e}")
            return {'happiness': 0.0, 'sadness': 0.0, 'anger': 0.0, 'surprise': 0.0, 'fear': 1.0}

        
    def test_emotion_detection(csv_path):
        test_data = pd.read_csv(csv_path)
    
        results = []
        print("\n--- Emotion Detection Results ---\n")
        for idx, row in test_data.iterrows():
            try:
                print(f"Processing record {idx + 1}...")
    
                # Face emotion detection
                face_emotion = detect_face_emotion_from_file(row['ImagePath'])
                print(f"Face Emotion Output: {face_emotion}")
    
                # LSTM emotion detection
                lstm_input = list(map(float, row['LSTMInput'].strip('"').split(',')))
                lstm_emotion = detect_lstm_emotion([lstm_input])
                print(f"LSTM Emotion Output: {lstm_emotion}")
    
                # Sleep emotion detection
                sleep_input = list(map(float, row['SleepInput'].strip('"').split(',')))
                sleep_emotion = detect_sleep_emotion(sleep_input)
                print(f"Sleep Emotion Output: {sleep_emotion}")
    
                # Map and Combine Results
                mapped_face = map_emotion_values(face_emotion, emotion_mappings['face'])
                mapped_lstm = map_emotion_values(lstm_emotion, emotion_mappings['lstm'])
                mapped_sleep = map_emotion_values(sleep_emotion, emotion_mappings['sleep'])
                combined = combine_emotion_values([mapped_face, mapped_lstm, mapped_sleep], [0.4, 0.4, 0.2])
                final_emotion = max(combined, key=combined.get)
                print(f"final_emotion Output: {final_emotion}")
                results.append([idx + 1, row['GroundTruthEmotion'], final_emotion, combined])
            except Exception as e:
                print(f"Error processing record {idx + 1}: {e}")
                results.append([idx + 1, row['GroundTruthEmotion'], "Error", {}])
    
        # Display Results
        headers = ["Record", "Ground Truth", "Predicted Emotion", "Combined Values"]
        #headers = ["Record", "Ground Truth", "Predicted Emotion", "Combined Values"]
        print(tabulate(results, headers=headers, tablefmt="grid"))
    
        # Calculate Accuracy
        predictions = [result[2] for result in results if result[2] != "Error"]
        ground_truths = [result[1] for result in results if result[2] != "Error"]
        accuracy = accuracy_score(ground_truths, predictions)
        print(f"\nFinal Accuracy: {accuracy:.2f}")

    def test_emotion_detection_readable(csv_path):
        test_data = pd.read_csv(csv_path)
    
        results = []
        for idx, row in test_data.iterrows():
            try:
                # Face emotion detection
                face_emotion = detect_face_emotion_from_file(row['ImagePath'])
                face_emotion_readable = {k: round(float(v), 2) for k, v in face_emotion.items()}
    
                # LSTM emotion detection
                lstm_input = list(map(float, row['LSTMInput'].strip('"').split(',')))
                lstm_emotion = detect_lstm_emotion([lstm_input])
                lstm_emotion_readable = {k: round(float(v), 2) for k, v in lstm_emotion.items()}
    
                # Sleep emotion detection
                sleep_input = list(map(float, row['SleepInput'].strip('"').split(',')))
                sleep_emotion = detect_sleep_emotion(sleep_input)
                sleep_emotion_readable = {k: round(float(v), 2) for k, v in sleep_emotion.items()}
                #print(f"Sleep Detection Output: {sleep_emotion_readable}")
    
                # Map and Combine Results
                mapped_face = map_emotion_values(face_emotion, emotion_mappings['face'])
                mapped_lstm = map_emotion_values(lstm_emotion, emotion_mappings['lstm'])
                mapped_sleep = map_emotion_values(sleep_emotion, emotion_mappings['sleep'])
                combined = combine_emotion_values([mapped_face, mapped_lstm, mapped_sleep], [0.4, 0.4, 0.1])
                final_emotion = max(combined, key=combined.get)
    
                results.append({
                    "Record": idx + 1,
                    "Ground Truth": row['GroundTruthEmotion'],
                    #"Face Model": face_emotion_readable,
                    #"LSTM Model": lstm_emotion_readable,
                    #"Sleep Model": sleep_emotion_readable,
                    "Predicted Emotion": final_emotion
                })
            except Exception as e:
                results.append({
                    "Record": idx + 1,
                    "Ground Truth": row['GroundTruthEmotion'],
                   # "Face Model": "Error",
                    #"LSTM Model": "Error",
                    #"Sleep Model": "Error",
                    "Predicted Emotion": "Error"
                })
    
        # Convert results to a DataFrame for a better display
        df_results = pd.DataFrame(results)
    
        # Display Results in a Clean Tabular Format
        print(tabulate(
            df_results,
            headers="keys",
            tablefmt="grid",
            showindex=False
        ))
    
        # Calculate and Display Accuracy
        valid_results = df_results[df_results["Predicted Emotion"] != "Error"]
        accuracy = accuracy_score(valid_results["Ground Truth"], valid_results["Predicted Emotion"])
        print(f"\nFinal Accuracy: {accuracy:.2f}")
    
    
    # Emotion Mappings
    emotion_mappings = {
        "face": {
            'Happy': 'Happy',
            'Sad': 'Stressed',
            'Neutral': 'Bored',
            'Angry': 'Stressed',
            'Surprise': 'Happy',  # Ensure 'Surprise' is included
            'Disgust': 'Stressed',     # Include 'Disgust' if face model outputs it
            'Fear': 'Stressed'
        },
        "lstm": {
            'Baseline': 'Bored',
            'Stress': 'Stressed',
            'Amusement': 'Happy',
            'Meditation': 'Happy'
        },
       "sleep": {'happiness': 'Happy', 'sadness': 'Stressed', 'anger': 'Stressed', 'surprise': 'Happy', 'Surprise': 'Happy', 'fear': 'Stressed'}
    }
    
    # Initialize models and test
    initialize_models()
    csv_path = 'test_data.csv'  # Update with your CSV file path
    test_emotion_detection_readable(csv_path)


Face emotion model loaded successfully.
LSTM model loaded successfully.
Sleep emotion model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━